In [1]:
# Import Modules
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.subplots as ms
import plotly.express as px
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import streamlit as st
from datetime import datetime, timedelta
from pykrx import stock
from pykrx import bond
import seaborn as sns
import scipy.optimize as opt
from scipy.optimize import minimize

import yfinance as yf

# plot 설정
plt.rc('font', family ='Malgun Gothic') # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False #마이너스 기호 깨짐 방지

In [2]:
# # 투자자 성향 입력
# risk_tolerance = st.selectbox("투자자 성향", ("공격적", "중립적", "방어적"))
# 안정형/안정추구형/위험중립형/적극투자형/공격투자형

# # 섹터별 ETF 목록
# 정보기술, 헬스케어, 소비재, 금융, 커뮤니케이션서비스, 산업재, 에너지, 철강, 건설
# # 주식 정보
# 에너지, 소재, 유틸리티, 정보기술, 건설, 헬스케어, 금융, 산업재, 소비재

# sector_etfs = {
#     "Technology": "XLK",
#     "Healthcare": "XLV",
#     "Financials": "XLF",
#     "Consumer Discretionary": "XLY",
#     "Consumer Staples": "XLP",
#     "Energy": "XLE",
#     "Industrials": "XLI",
#     "Materials": "XLB",
#     "Utilities": "XLU",
#     "Real Estate": "XLRE"
# }

# 포트폴리오 구성 비율

- 섹터별 ETF 목록
: 정보기술, 헬스케어, 소비재, 금융, 커뮤니케이션서비스, 산업재, 에너지, 철강, 건설
- 주식 정보
: 에너지, 소재, 유틸리티, 정보기술, 건설, 헬스케어, 금융, 산업재, 소비재

## 단순 
- risk_aversion_factor는 투자자의 위험 회피 성향을 반영하며, 높을수록 안정성을 추구

In [3]:
# 예시로 사용할 ETF 티커 리스트
etf_tickers = ['XLK', 'XLV', 'XLY', 'XLF', 'XLC', 'XLI', 'XLE', 'SLX', 'ITB']

# 데이터 다운로드
data = yf.download(etf_tickers, start='2020-01-01', end='2023-01-01')['Adj Close']

# 일별 수익률 계산
returns = data.pct_change().dropna()


[*********************100%***********************]  9 of 9 completed


In [4]:
# 변동성 (연율화된 표준편차) 계산
volatility = returns.std() * np.sqrt(252)

In [5]:
def investor_preference(volatility, risk_aversion_factor):
    # 위험 회피 계수에 따라 가중치 계산
    inverse_volatility = 1 / volatility
    weights = inverse_volatility / inverse_volatility.sum()
    
    # 위험 회피 계수 적용 (높을수록 안전한 자산에 비중 증가)
    weights = weights ** risk_aversion_factor
    weights = weights / weights.sum()
    return weights

In [6]:
def portfolio_volatility(weights, returns):
    portfolio_return = np.dot(weights, returns.mean() * 252)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    return portfolio_volatility

# 제약 조건과 초기 가중치 설정
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(etf_tickers)))
initial_weights = np.array([1/len(etf_tickers)] * len(etf_tickers))

# 최적화 수행
optimal_weights = minimize(portfolio_volatility, initial_weights, args=(returns,),
                           method='SLSQP', bounds=bounds, constraints=constraints)

# 결과 가중치
optimal_weights.x


array([1.48717202e-18, 0.00000000e+00, 1.05494631e-01, 1.40568769e-17,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.94505369e-01,
       0.00000000e+00])

In [7]:
portfolio = pd.DataFrame({'ETF': etf_tickers, 'Weight': optimal_weights.x})
print(portfolio)


   ETF        Weight
0  XLK  1.487172e-18
1  XLV  0.000000e+00
2  XLY  1.054946e-01
3  XLF  1.405688e-17
4  XLC  0.000000e+00
5  XLI  0.000000e+00
6  XLE  0.000000e+00
7  SLX  8.945054e-01
8  ITB  0.000000e+00


## (마코비츠) 포트폴리오 이론 기반 = 샤프지수, 변동성

In [9]:
# 포트폴리오 종목들, 각 비중, 매수일, 매도일 입력시 백테스팅한 결과를 반환하는 함수 : (로그)수익률, 분산(표준편차), 일별 종가 df를 반환
def back_test(codes, weight, start_date, end_date):
    # 포트폴리오 df 생성
    portfolio_df = pd.DataFrame()
    for code in codes :
        temp = stock.get_market_ohlcv(start_date,end_date, code, adjusted = True)[['종가']] ; time.sleep(0.5)
        portfolio_df =  pd.concat([portfolio_df, temp], axis = 1)
    portfolio_df.columns = codes

    # 로그 수익률 -> 이걸 쓰는게 맞다
    log_ret = portfolio_df.apply(np.log).diff().dropna()
    log_returns = log_ret.cumsum(axis = 0).iloc[-1]
    portfolioLogReturn = np.sum(log_returns*weight)

    # 변동성 위험
    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance* 100, 2)) + '%'
    percent_vols = str(round(port_volatility* 100, 2)) + '%'
    percent_ret = str(round(portfolioLogReturn*100, 2))+'%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var

#평균수익률과 표준편차 반환하기 위한 함수
def ret_std(weight, ret, log_ret): 
    port_mean = np.sum(weight * ret.mean() *len(log_ret))               #포트폴리오 수익률의 평균구하기
    port_var = np.dot(weight.T, np.dot(ret.cov()*len(log_ret), weight)) #포트폴리오 수익률의 분산구하기
    port_std = np.sqrt(port_var)                                        #분산에 루트를 취해 표준편차 구한 모습
    return port_mean, port_std                                         #평균수익률과 표준편차 반환

#포트폴리오 수익률과 volatility, sharpe ratio반환하는 함수
def statistics(weights,ret, log_ret, rf=0): # statistics 선언(포트폴리오 수익률, volatility, sharpe ratio반환)
    weights = np.array(weights)             #weight를 array변환
    pret = np.sum(ret.mean() * weights) * len(log_ret) - rf  #포트폴리오 수익률 생성
    pvol = np.sqrt(np.dot(weights.T, np.dot(ret.cov() * len(log_ret), weights))) #포트폴리오 volatility 생성
    return np.array([pret, pvol, pret / pvol])  #포트폴리오 수익률, volatility, sharpe ratio반환

#-sharpe ratio 반환하는 함수 
def min_func_sharpe(weights,ret, log_ret, rf=0): 
    return -statistics(weights,ret, log_ret, rf)[2]

#volatility^2 반환하는 함수
def min_func_volatility(weights,ret, log_ret): 
    return statistics(weights,ret, log_ret)[1] **2 

# 랜덤하게 5000개의 포트폴리오를 생성하고 구성종목의 비중을 랜덤하게 부여하는 함수 : 최적포트폴리오 탐색 및 시각화 목적으로 생성
def make_random_portfolio(ret, noa, log_ret):
    port_rets = [] ; port_std = []
    for w in range(5000): #랜덤하게 나머지 비중을 부여하여 5000개 포트만들기
        weight = np.random.random(noa) #종목개수만큼 랜덤한 숫자를 부여함.(랜덤하게 비중 부여하는 작업)
        weight /= np.sum(weight) #포트전체비율에서 5종목이 랜덤하게 나누어 가지도록 부여
        mu, sig = ret_std(np.array(weight), ret, log_ret) #ret_std함수로 수익률의 평균과 표준편차 반환
        port_rets.append(mu) #ret 리스트에 수익률 평균 append
        port_std.append(sig) #std 리스트에 수익률 표준편차 append
    return port_rets, port_std

# 포트폴리오이론을 기반으로 최적포트폴리오 그래프, 비율, 샤프지수 등을 반환하는 함수
def portfolio_opt(port_rets, port_std, noa, ret, portfolio_df, log_ret, end_date):
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))
    # plt.style.use('seaborn')

    #다음은 최적화 함수에 들어가는 argument들을 모아놓은 것임
    cons = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})  #가중치의 합이 1이되도록(100%)가 되도록 제약
    bnds = tuple((0,1) for i in range((noa)))   # ((0, 1), (0, 1), (0, 1), (0, 1)) 0~1까지의 값을 가질 수 있음
    k = noa * [1. / noa,] # initial guess 

    #다음은 -sharpe ratio를 최소화하는 작업임(sharpe ratio 최대화)
    opts = opt.minimize(min_func_sharpe, k, method='SLSQP',
                           bounds=bnds, constraints=cons, args=(ret, log_ret))
    #다음은 volatility^2를 최소화하는 작업임
    optv = opt.minimize(min_func_volatility, k, method='SLSQP',
                           bounds=bnds, constraints=cons, args=(ret, log_ret))

    axes[0].scatter(port_std, port_rets, #x축에 포트폴리오 표준편차, y축에 평균 수익률을 점(.)으로 표시
                c=np.array(port_rets) / np.array(port_std), marker='.', cmap = 'Greys')#  sr을 좌측에 하양,회색의 cmap으로 표시
                # random portfolio composition

    axes[0].plot(statistics(opts['x'],ret, log_ret)[1], statistics(opts['x'],ret, log_ret)[0], #sharpe ratio가 가장 높은 port를 빨간 별
             'r*', markersize=15.0, label = 'Portfolio with highest Sharpe Ratio') #라벨링 'Portfolio with highest Sharpe Ratio'
                # portfolio with highest Sharpe ratio
    axes[0].plot(statistics(optv['x'],ret, log_ret)[1], statistics(optv['x'],ret, log_ret)[0],#variance가 가장 낮은 port를 노란 별
             'y*', markersize=15.0, label = 'Minimum variance portfolio') #라벨링 'Minimum variance portfolio'
                # minimum variance portfolio

    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_xlabel('variance')
    axes[0].set_ylabel('expected return')

    rf = bond.get_otc_treasury_yields(end_date.strftime('%Y%m%d')).iloc[3,0]/100  #risk free asset 수익률: 10년물 국채 수익률 실시간 스크래핑
    slope = (statistics(opts['x'],ret, log_ret)[0] - rf) / statistics(opts['x'],ret, log_ret)[1] #mean variance frontier의 기울기 구하기
    x =  np.linspace(0.0,0.24,5000) #0.0부터 0.24까지 일정간격으로 5000개 x 생성
    y = [x*slope + rf for x in np.linspace(0.0,0.24,5000)]#mean variance frontier의 y값들 설정(x에 기울기 곱하고 rf더한 값들)


    axes[1].scatter(port_std, port_rets,  #x축에 포트폴리오 표준편차, y축에 평균 수익률을 점(.)으로 표시
                c=np.array(port_rets) / np.array(port_std), marker='.',cmap = 'Greys') #  sr을 좌측에 흰색,회색의 cmap으로 표시  

    axes[1].plot(x,y, label = 'mean-variance frontier with riskfree asset') #mean variance frontier를 그린다. 
    axes[1].plot(statistics(opts['x'],ret, log_ret)[1], statistics(opts['x'],ret, log_ret)[0],#sharpe ratio가 가장 높은 port를 빨간 별
             'r*', markersize=15.0, label = 'Portfolio with highest Sharpe Ratio')#라벨링 'Portfolio with highest Sharpe Ratio'
                # portfolio with highest Sharpe ratio

    axes[1].legend() #상단에 범례 출력
    axes[1].grid(True) # 격자 생성
    axes[1].set_xlabel('expected volatility')#x축에 라벨링 'expected volatility'     
    axes[1].set_ylabel('expected return') #y축에 라벨링 'expected return'     
    
    sharpe_ratio = statistics(opts['x'],ret, log_ret,rf)[2]
    opt_weight = {portfolio_df.columns[i] : round(opts['x'][i],10)  for i in range(noa)}
    
    return fig, sharpe_ratio, opt_weight

sns.set()
plt.rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# 포트폴리오 종목들, 각 비중, 매수일, 매도일 입력시 백테스팅한 결과를 반환하는 함수
def back_test(codes, weight, start_date, end_date):
    portfolio_df = pd.DataFrame()
    for code in codes:
        temp = stock.get_market_ohlcv(start_date, end_date, code, adjusted=True)[['종가']]
        time.sleep(0.5)
        portfolio_df = pd.concat([portfolio_df, temp], axis=1)
    portfolio_df.columns = codes

    log_ret = portfolio_df.apply(np.log).diff().dropna()
    log_returns = log_ret.cumsum(axis=0).iloc[-1]
    portfolioLogReturn = np.sum(log_returns * weight)

    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance * 100, 2)) + '%'
    percent_vols = str(round(port_volatility * 100, 2)) + '%'
    percent_ret = str(round(portfolioLogReturn * 100, 2)) + '%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var

# 평균수익률과 표준편차 반환하기 위한 함수
def ret_std(weight, ret, log_ret):
    port_mean = np.sum(weight * ret.mean() * len(log_ret))
    port_var = np.dot(weight.T, np.dot(ret.cov() * len(log_ret), weight))
    port_std = np.sqrt(port_var)
    return port_mean, port_std

# 포트폴리오 수익률과 volatility, sharpe ratio 반환하는 함수
def statistics(weights, ret, log_ret, rf=0):
    weights = np.array(weights)
    pret = np.sum(ret.mean() * weights) * len(log_ret) - rf
    pvol = np.sqrt(np.dot(weights.T, np.dot(ret.cov() * len(log_ret), weights)))
    return np.array([pret, pvol, pret / pvol])

# -sharpe ratio 반환하는 함수
def min_func_sharpe(weights, ret, log_ret, rf=0):
    return -statistics(weights, ret, log_ret, rf)[2]

# volatility^2 반환하는 함수
def min_func_volatility(weights, ret, log_ret):
    return statistics(weights, ret, log_ret)[1] ** 2

# 랜덤하게 5000개의 포트폴리오를 생성하고 구성종목의 비중을 랜덤하게 부여하는 함수
def make_random_portfolio(ret, noa, log_ret):
    port_rets = []
    port_std = []
    for _ in range(5000):
        weight = np.random.random(noa)
        weight /= np.sum(weight)
        mu, sig = ret_std(np.array(weight), ret, log_ret)
        port_rets.append(mu)
        port_std.append(sig)
    return port_rets, port_std

# 포트폴리오 최적화 및 시각화 함수
def portfolio_opt(port_rets, port_std, noa, ret, portfolio_df, log_ret, end_date):
    fig, axes = plt.subplots(1, 2, figsize=(18, 6))

    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bnds = tuple((0, 1) for _ in range(noa))
    k = noa * [1. / noa,]

    opts = opt.minimize(min_func_sharpe, k, method='SLSQP', bounds=bnds, constraints=cons, args=(ret, log_ret))
    optv = opt.minimize(min_func_volatility, k, method='SLSQP', bounds=bnds, constraints=cons, args=(ret, log_ret))

    axes[0].scatter(port_std, port_rets, c=np.array(port_rets) / np.array(port_std), marker='.', cmap='Greys')
    axes[0].plot(statistics(opts['x'], ret, log_ret)[1], statistics(opts['x'], ret, log_ret)[0], 'r*', markersize=15.0, label='Portfolio with highest Sharpe Ratio')
    axes[0].plot(statistics(optv['x'], ret, log_ret)[1], statistics(optv['x'], ret, log_ret)[0], 'y*', markersize=15.0, label='Minimum variance portfolio')
    axes[0].legend()
    axes[0].grid(True)
    axes[0].set_xlabel('variance')
    axes[0].set_ylabel('expected return')

    rf = bond.get_otc_treasury_yields(end_date.strftime('%Y%m%d')).iloc[3, 0] / 100
    slope = (statistics(opts['x'], ret, log_ret)[0] - rf) / statistics(opts['x'], ret, log_ret)[1]
    x = np.linspace(0.0, 0.24, 5000)
    y = [i * slope + rf for i in x]

    axes[1].scatter(port_std, port_rets, c=np.array(port_rets) / np.array(port_std), marker='.', cmap='Greys')
    axes[1].plot(x, y, label='mean-variance frontier with riskfree asset')
    axes[1].plot(statistics(opts['x'], ret, log_ret)[1], statistics(opts['x'], ret, log_ret)[0], 'r*', markersize=15.0, label='Portfolio with highest Sharpe Ratio')
    axes[1].legend()
    axes[1].grid(True)
    axes[1].set_xlabel('expected volatility')
    axes[1].set_ylabel('expected return')

    sharpe_ratio = statistics(opts['x'], ret, log_ret, rf)[2]
    opt_weight = {portfolio_df.columns[i]: round(opts['x'][i], 10) for i in range(noa)}

    return fig, sharpe_ratio, opt_weight


In [15]:
def markobitz(codes, weight, start_date, end_date):
    portfolio_df = pd.DataFrame()
    for code in codes:
        temp = yf.download(code, start=start_date, end=end_date)['Adj Close']
        time.sleep(0.5)
        portfolio_df = pd.concat([portfolio_df, temp], axis=1)
    portfolio_df.columns = codes

    # 로그 수익률 계산
    log_ret = portfolio_df.apply(np.log).diff().dropna()
    log_returns = log_ret.cumsum(axis=0).iloc[-1]
    portfolio_log_return = np.sum(log_returns * weight)

    # 변동성 계산
    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance * 100, 2)) + '%'
    percent_vols = str(round(port_volatility * 100, 2)) + '%'
    percent_ret = str(round(portfolio_log_return * 100, 2)) + '%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var


In [17]:
def markowitz_portfolio(codes, start_date, end_date):
    portfolio_df = pd.DataFrame()
    for code in codes:
        temp = yf.download(code, start=start_date, end=end_date)['Adj Close']
        time.sleep(0.5)
        portfolio_df = pd.concat([portfolio_df, temp], axis=1)
    portfolio_df.columns = codes

    # 로그 수익률 계산
    log_ret = portfolio_df.apply(np.log).diff().dropna()
    
    noa = len(codes)
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bnds = tuple((0, 1) for _ in range(noa))
    init_guess = noa * [1. / noa,]

    def statistics(weights, ret, log_ret):
        weights = np.array(weights)
        pret = np.sum(ret.mean() * weights) * len(log_ret)
        pvol = np.sqrt(np.dot(weights.T, np.dot(ret.cov() * len(log_ret), weights)))
        return np.array([pret, pvol, pret / pvol])

    def min_func_sharpe(weights, ret, log_ret, rf=0):
        return -statistics(weights, ret, log_ret)[2]

    def min_func_volatility(weights, ret, log_ret):
        return statistics(weights, ret, log_ret)[1] ** 2

    # 샤프 지수 최대화 포트폴리오
    opts = minimize(min_func_sharpe, init_guess, method='SLSQP', bounds=bnds, constraints=cons, args=(log_ret, log_ret))

    weight = opts.x
    log_returns = log_ret.cumsum(axis=0).iloc[-1]
    portfolio_log_return = np.sum(log_returns * weight)

    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance * 100, 2)) + '%'
    percent_vols = str(round(port_volatility * 100, 2)) + '%'
    percent_ret = str(round(portfolio_log_return * 100, 2)) + '%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var, weight


## 등가중 포트폴리오

In [10]:
def equal_weight_portfolio(codes, start_date, end_date):
    # 데이터 가져오기
    portfolio_df = pd.DataFrame()
    for code in codes:
        temp = yf.download(code, start=start_date, end=end_date)['Adj Close']
        time.sleep(0.5)
        portfolio_df = pd.concat([portfolio_df, temp], axis=1)
    
    portfolio_df.columns = codes

    # 등가중 포트폴리오
    weight = np.array([1/len(codes)] * len(codes))

    # 로그 수익률 계산
    log_ret = portfolio_df.apply(np.log).diff().dropna()
    log_returns = log_ret.cumsum(axis=0).iloc[-1]
    portfolio_log_return = np.sum(log_returns * weight)

    # 변동성 계산
    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance * 100, 2)) + '%'
    percent_vols = str(round(port_volatility * 100, 2)) + '%'
    percent_ret = str(round(portfolio_log_return * 100, 2)) + '%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var


## 최소분산 포트폴리오

In [11]:
def min_variance_portfolio(codes, start_date, end_date):
    # 데이터 가져오기
    portfolio_df = pd.DataFrame()
    for code in codes:
        temp = yf.download(code, start=start_date, end=end_date)['Adj Close']
        time.sleep(0.5)
        portfolio_df = pd.concat([portfolio_df, temp], axis=1)
    
    portfolio_df.columns = codes

    # 로그 수익률 계산
    log_ret = portfolio_df.apply(np.log).diff().dropna()
    
    noa = len(codes)
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bnds = tuple((0, 1) for _ in range(noa))
    init_guess = noa * [1. / noa,]

    # 최적화
    optv = minimize(min_func_volatility, init_guess, method='SLSQP', bounds=bnds, constraints=cons, args=(log_ret, log_ret))

    weight = optv.x
    log_returns = log_ret.cumsum(axis=0).iloc[-1]
    portfolio_log_return = np.sum(log_returns * weight)

    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance * 100, 2)) + '%'
    percent_vols = str(round(port_volatility * 100, 2)) + '%'
    percent_ret = str(round(portfolio_log_return * 100, 2)) + '%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var, weight


##  최대 샤프지수 포트폴리오

In [12]:
def max_sharpe_portfolio(codes, start_date, end_date, rf=0.02):
    # 데이터 가져오기
    portfolio_df = pd.DataFrame()
    for code in codes:
        temp = yf.download(code, start=start_date, end=end_date)['Adj Close']
        time.sleep(0.5)
        portfolio_df = pd.concat([portfolio_df, temp], axis=1)
    
    portfolio_df.columns = codes

    # 로그 수익률 계산
    log_ret = portfolio_df.apply(np.log).diff().dropna()
    
    noa = len(codes)
    cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bnds = tuple((0, 1) for _ in range(noa))
    init_guess = noa * [1. / noa,]

    # 최적화
    opts = minimize(min_func_sharpe, init_guess, method='SLSQP', bounds=bnds, constraints=cons, args=(log_ret, log_ret, rf))

    weight = opts.x
    log_returns = log_ret.cumsum(axis=0).iloc[-1]
    portfolio_log_return = np.sum(log_returns * weight)

    cov_matrix = log_ret.cov() * len(log_ret)
    port_variance = np.dot(weight.T, np.dot(cov_matrix, weight))
    port_volatility = np.sqrt(port_variance)

    percent_var = str(round(port_variance * 100, 2)) + '%'
    percent_vols = str(round(port_volatility * 100, 2)) + '%'
    percent_ret = str(round(portfolio_log_return * 100, 2)) + '%'

    return log_ret, portfolio_df, percent_ret, percent_vols, percent_var, weight


## 사용예시

### 

In [16]:
# ETF 목록
etf_tickers = ['XLK', 'XLV', 'XLY', 'XLF', 'XLC', 'XLI', 'XLE', 'SLX', 'ITB']
start_date = '2020-01-01'
end_date = '2023-01-01'

# 마코비츠 포트폴리오
weights = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])  # 예시 가중치
log_ret_m, portfolio_df_m, percent_ret_m, percent_vols_m, percent_var_m = markobitz(etf_tickers, weights, start_date, end_date)
print(f"Markowitz Portfolio: Return = {percent_ret_m}, Volatility = {percent_vols_m}, Variance = {percent_var_m}")

# 등가중 포트폴리오
log_ret_eq, portfolio_df_eq, percent_ret_eq, percent_vols_eq, percent_var_eq = equal_weight_portfolio(etf_tickers, start_date, end_date)
print(f"Equal Weight Portfolio: Return = {percent_ret_eq}, Volatility = {percent_vols_eq}, Variance = {percent_var_eq}")

# 최소 분산 포트폴리오
log_ret_mv, portfolio_df_mv, percent_ret_mv, percent_vols_mv, percent_var_mv, weight_mv = min_variance_portfolio(etf_tickers, start_date, end_date)
print(f"Minimum Variance Portfolio: Return = {percent_ret_mv}, Volatility = {percent_vols_mv}, Variance = {percent_var_mv}, Weights = {weight_mv}")

# 최대 샤프 지수 포트폴리오
log_ret_sr, portfolio_df_sr, percent_ret_sr, percent_vols_sr, percent_var_sr, weight_sr = max_sharpe_portfolio(etf_tickers, start_date, end_date)
print(f"Maximum Sharpe Ratio Portfolio: Return = {percent_ret_sr}, Volatility = {percent_vols_sr}, Variance = {percent_var_sr}, Weights = {weight_sr}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Markowitz Portfolio: Return = 23.83%, Volatility = 43.95%, Variance = 19.31%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%

### 

In [13]:
# ETF 목록
etf_tickers = ['XLK', 'XLV', 'XLY', 'XLF', 'XLC', 'XLI', 'XLE', 'SLX', 'ITB']
start_date = '2020-01-01'
end_date = '2023-01-01'

# 등가중 포트폴리오
log_ret_eq, portfolio_df_eq, percent_ret_eq, percent_vols_eq, percent_var_eq = equal_weight_portfolio(etf_tickers, start_date, end_date)
print(f"Equal Weight Portfolio: Return = {percent_ret_eq}, Volatility = {percent_vols_eq}, Variance = {percent_var_eq}")

# 최소 분산 포트폴리오
log_ret_mv, portfolio_df_mv, percent_ret_mv, percent_vols_mv, percent_var_mv, weight_mv = min_variance_portfolio(etf_tickers, start_date, end_date)
print(f"Minimum Variance Portfolio: Return = {percent_ret_mv}, Volatility = {percent_vols_mv}, Variance = {percent_var_mv}, Weights = {weight_mv}")

# 최대 샤프 지수 포트폴리오
log_ret_sr, portfolio_df_sr, percent_ret_sr, percent_vols_sr, percent_var_sr, weight_sr = max_sharpe_portfolio(etf_tickers, start_date, end_date)
print(f"Maximum Sharpe Ratio Portfolio: Return = {percent_ret_sr}, Volatility = {percent_vols_sr}, Variance = {percent_var_sr}, Weights = {weight_sr}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Equal Weight Portfolio: Return = 26.48%, Volatility = 48.83%, Variance = 23.84%
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************1

### 

In [18]:
# ETF 목록
etf_tickers = ['XLK', 'XLV', 'XLY', 'XLF', 'XLC', 'XLI', 'XLE', 'SLX', 'ITB']
start_date = '2020-01-01'
end_date = '2023-01-01'

# 마코비츠 포트폴리오
log_ret_m, portfolio_df_m, percent_ret_m, percent_vols_m, percent_var_m, weight_m = markowitz_portfolio(etf_tickers, start_date, end_date)
print(f"Markowitz Portfolio: Return = {percent_ret_m}, Volatility = {percent_vols_m}, Variance = {percent_var_m}, Weights = {weight_m}")

# 등가중 포트폴리오
log_ret_eq, portfolio_df_eq, percent_ret_eq, percent_vols_eq, percent_var_eq = equal_weight_portfolio(etf_tickers, start_date, end_date)
print(f"Equal Weight Portfolio: Return = {percent_ret_eq}, Volatility = {percent_vols_eq}, Variance = {percent_var_eq}")

# 최소 분산 포트폴리오
log_ret_mv, portfolio_df_mv, percent_ret_mv, percent_vols_mv, percent_var_mv, weight_mv = min_variance_portfolio(etf_tickers, start_date, end_date)
print(f"Minimum Variance Portfolio: Return = {percent_ret_mv}, Volatility = {percent_vols_mv}, Variance = {percent_var_mv}, Weights = {weight_mv}")

# 최대 샤프 지수 포트폴리오
log_ret_sr, portfolio_df_sr, percent_ret_sr, percent_vols_sr, percent_var_sr, weight_sr = max_sharpe_portfolio(etf_tickers, start_date, end_date)
print(f"Maximum Sharpe Ratio Portfolio: Return = {percent_ret_sr}, Volatility = {percent_vols_sr}, Variance = {percent_var_sr}, Weights = {weight_sr}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Markowitz Portfolio: Return = 39.31%, Volatility = 41.6%, Variance = 17.31%, Weights = [1.42019479e-16 7.22992734e-01 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 4.33233618e-02 2.33683905e-01
 5.83074808e-18]
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**************

# 섹터별 기업 추천
: 섹터별 기업 정보 DB (가정) 에서 기투자 종목 보다 나은 or 추천 종목 == "영업이익 high, PER low. ROE high.부채비율 low" 기준 

## 섹터 내 기업 매칭 (DB 연결)

In [21]:
INFO = pd.read_json("C:/Users/koby0/OneDrive/바탕 화면/info.json")

In [23]:
info = INFO[INFO['kospi200_name_code'].notna()]
info

,kospi200_name_code,kospi200_code_name,event_dict
삼성전자,5930.0,NaN,NaN
LG에너지솔루션,373220.0,NaN,NaN
SK하이닉스,660.0,NaN,NaN
삼성바이오로직스,207940.0,NaN,NaN
LG화학,51910.0,NaN,NaN
...,...,...,...
지누스,13890.0,NaN,NaN
쿠쿠홈시스,284740.0,NaN,NaN
현대홈쇼핑,57050.0,NaN,NaN
한섬,20000.0,NaN,NaN


In [42]:
info_reset = info.reset_index()
info_reset

,index,kospi200_name_code,kospi200_code_name,event_dict
0,삼성전자,5930.0,NaN,NaN
1,LG에너지솔루션,373220.0,NaN,NaN
2,SK하이닉스,660.0,NaN,NaN
3,삼성바이오로직스,207940.0,NaN,NaN
4,LG화학,51910.0,NaN,NaN
...,...,...,...,...
195,지누스,13890.0,NaN,NaN
196,쿠쿠홈시스,284740.0,NaN,NaN
197,현대홈쇼핑,57050.0,NaN,NaN
198,한섬,20000.0,NaN,NaN


In [28]:
# 모든 시트를 데이터 프레임으로 읽기
excel_data = pd.read_excel("C:/Users/koby0/OneDrive/바탕 화면/주식정보.xlsx", sheet_name=None)

# 각 시트에 시트 이름을 새로운 열로 추가
for sheet_name, df in excel_data.items():
    df['SheetName'] = sheet_name
# 모든 시트를 하나의 데이터 프레임으로 합치기
combined_df = pd.concat(excel_data.values(), ignore_index=True)

# 결과 출력
#print(combined_df)


     No.      종목명      기준 시장      자산총계     자본금      자본총계       매출액     영업이익  \
0      1  SK이노베이션  202312  유  21362775  509520  18219710   2489430  1235472   
1      2    S-Oil  202312  유  21498548  291512   9025125  35251031  1350825   
2      3       GS  202312  유   6934380  473501   6197061   1004792   918375   
3      4     HD현대  202312  유   9122274   81433   5929776    583102   487637   
4      5     SK가스  202312  유   5034185   46151   2362882   5290044   318426   
..   ...      ...     ... ..       ...     ...       ...       ...      ...   
556   30   오리온홀딩스  202312  유   2008816   31323   1840506     33674    20806   
557   31     마녀공장  202312  코    105871    1638     97491    102292    15892   
558   32     잇츠한불  202312  유    392445   10965    385922     37964   -14046   
559   33      풀무원  202312  유    593547   21078    394666    126032    24639   
560   34      메지온  202312  코     75078   14804     61090     31721    -3521   

         순이익 SheetName Unnamed: 10  Unnamed: 0  Unn

In [32]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No.          561 non-null    int64  
 1   종목명          561 non-null    object 
 2   기준           561 non-null    int64  
 3   시장           561 non-null    object 
 4   자산총계         561 non-null    int64  
 5   자본금          561 non-null    int64  
 6   자본총계         561 non-null    int64  
 7   매출액          561 non-null    int64  
 8   영업이익         561 non-null    int64  
 9   순이익          561 non-null    int64  
 10  SheetName    561 non-null    object 
 11  Unnamed: 10  6 non-null      object 
 12  Unnamed: 0   0 non-null      float64
 13  Unnamed: 1   0 non-null      float64
 14  Unnamed: 2   0 non-null      float64
dtypes: float64(3), int64(8), object(4)
memory usage: 65.9+ KB


In [35]:
stock_info = combined_df[["종목명", "시장", "SheetName"]]

In [36]:
stock_info

,종목명,시장,SheetName
0,SK이노베이션,유,에너지
1,S-Oil,유,에너지
2,GS,유,에너지
3,HD현대,유,에너지
4,SK가스,유,에너지
...,...,...,...
556,오리온홀딩스,유,소비재
557,마녀공장,코,소비재
558,잇츠한불,유,소비재
559,풀무원,유,소비재


In [43]:
# 데이터 프레임 병합
merged_df = pd.merge(stock_info, info_reset[['index', 'kospi200_name_code']], how='left', left_on='종목명', right_on='index')

# 'index' 열 삭제 (필요에 따라)
merged_df.drop(columns=['index'], inplace=True)

In [45]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 561 entries, 0 to 560
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   종목명                 561 non-null    object 
 1   시장                  561 non-null    object 
 2   SheetName           561 non-null    object 
 3   kospi200_name_code  219 non-null    float64
dtypes: float64(1), object(3)
memory usage: 21.9+ KB


## 기업분석

In [7]:
# 날짜와 기간(N일)을 입력하면 해당 날짜로 부터 N일 전의 날짜를 반환
def change_date(date , period):
    new_date = datetime.strptime(date, '%Y%m%d')
    new_date = new_date - timedelta(days=period)
    new_date = new_date.strftime('%Y%m%d')
    return new_date

# 검색시작일, 검색종료일, 종목코드를 입력 받아 주가 차트를 그리기 위한 정보들을 긁어오는 함수
def scrapping_info(start_date,end_date, code):
    #시가, 종가, 저가, 고가
    temp1 = stock.get_market_ohlcv(start_date,end_date, code, adjusted = True); time.sleep(0.5) 
    
    #펀더멘탈(BPS,EPS,PER,PBR,DPS,DIV)
    temp2 = stock.get_market_fundamental(start_date, end_date, code) ; time.sleep(0.5)
    
    #외국인 소진율
    temp3 = stock.get_exhaustion_rates_of_foreign_investment(start_date, end_date, code)[['한도소진률']] 
    
    # 시가총액 
    merged_df = temp1.merge(temp2, left_index=True, right_index=True, how='outer')
    merged_df = merged_df.merge(temp3, left_index=True, right_index=True, how='outer')
    merged_df['시가총액'] = ''
    nearest = stock.get_nearest_business_day_in_a_week(end_date)
    idx = datetime.strptime(nearest, '%Y%m%d').strftime('%Y-%m-%d')
    cap = stock.get_market_cap(nearest).loc[code, '시가총액']
    merged_df.loc[idx,'시가총액'] = cap
    
    # 5, 20, 60일 이동평균
    merged_df['5일_이동평균'] = merged_df['종가'].rolling(window=5).mean()
    merged_df['20일_이동평균'] = merged_df['종가'].rolling(window=20).mean()
    merged_df['60일_이동평균'] = merged_df['종가'].rolling(window=60).mean()
    
    return merged_df

# 앞선 scrapping_info를 활용하여 검색기간이 길어 주가 정보불러오는 것이 문제가 있을 경우 최근 한달의 정보만 가져오도록 예외처리하는 함수
def making_stock_df(start_date,end_date,code):
    try : 
        start_date1 = change_date(start_date, 90) #시작일 90일 전부터 시작하는 이유는 60개장일전의 정보를 이용해 이동평균선을 그리기 위함
        merged_df = scrapping_info(start_date1,end_date, code)
    except : 
        st.write("검색하신 기간이 길어 주가 정보를 불러올 수 없으므로 최근 한달의 정보를 불러옵니다")
        start_date = change_date(end_date, 120) #종료일 120일 전부터 시작하는 이유는 한달의 정보를 불러오되 60개장일 이동평균을 계산하기 위함
        merged_df = scrapping_info(start_date,end_date, code)
        start_date = change_date(end_date, 30)
        
    stock_df = merged_df.sort_index(ascending=False)
    ago = stock.get_nearest_business_day_in_a_week(start_date)
    stock_df = stock_df[stock_df.index > ago]
    return stock_df #stock_df.iloc[n] -> n 개장일 전의 정보

# 주가 dataframe을 입력받아 캔들차트를 그려 그림 객체를 반환하는 함수
def plot_candle(stock_df):
    # 캔들 차트 객체 생성
    candle = go.Candlestick(
        x=stock_df.index,
        open=stock_df['시가'],
        high=stock_df['고가'],
        low=stock_df['저가'],
        close=stock_df['종가'],
        increasing_line_color = 'red', # 상승봉 스타일링
        decreasing_line_color = 'blue', # 하락봉 스타일링
        name = 'Price'
    )

    # 스무스한 이동평균선 객체 생성(5, 20, 60일 이동평균선)
    ma_5 = go.Scatter(
        x=stock_df.index,
        y=stock_df['5일_이동평균'],
        mode='lines',
        line=dict(color="#98FB98", width=2),
        name=f'{5}Days Moving Average'
    )
    ma_20 = go.Scatter(
        x=stock_df.index,
        y=stock_df['20일_이동평균'],
        mode='lines',
        line=dict(color='#FF9999', width=2),
        name=f'{20}Days Moving Average'
    )
    ma_60 = go.Scatter(
        x=stock_df.index,
        y=stock_df['60일_이동평균'],
        mode='lines',
        line=dict(color='#FFB347', width=2),
        name=f'{60}Days Moving Average'
    )

    # 바 차트(거래량) 객체 생성
    volume_bar = go.Bar(x=stock_df.index, 
                        y=stock_df['거래량'], 
                        name = 'Volume',
                        marker=dict(color='#D3D3D3'))

    fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[3,1])

    fig.add_trace(candle, row=1, col=1)
    fig.add_trace(ma_5, row=1, col=1)
    fig.add_trace(ma_20, row=1, col=1)
    fig.add_trace(ma_60, row=1, col=1)
    fig.add_trace(volume_bar, row=2, col=1)

    fig.update_layout(
        # title='??',
        yaxis1_title='Stock Price',
        yaxis2_title='Volume',
        # xaxis2_title='periods',
        xaxis1_rangeslider_visible=False,
        xaxis2_rangeslider_visible=False,    
    )
    # 여백 설정
    fig.update_layout(
        margin=dict(t=0),  # 상단, 하단, 좌측, 우측 여백 값을 0으로 설정
        legend=dict(orientation='h')
    )
    return fig

# 금액 출력 형식 변환 코드 : 숫자를 입력받아 '조, 억, 만, 원' 중 2개까지 표현하는 문자열로 반환하는 함수
def format_currency(number): 
    if number < 10000:
        return f"{number}원"

    units = ["조", "억", "만"]
    scales = [1000000000000, 100000000, 10000]

    result = ""
    for unit, scale in zip(units, scales):
        if number >= scale:
            quotient = number // scale
            number %= scale
            result += f"{quotient:d}{unit} "

    result += f"{number:d}원"
    test = re.findall("(\d+.\s\d+[\w])",result)
    if test: return test[0]
    else : return result
    return result


def get_finance_info(code):
    url = f'https://finance.naver.com/item/main.naver?code={code}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    financial_section = soup.find('div', class_='section cop_analysis')
    if not financial_section:
        return None, "재무 섹션을 찾을 수 없습니다."

    rows = financial_section.find_all('tr')
    data = {}
    titles = []
    
    for row in rows:
        header = row.find('th')
        if header and header.get_text() in ["매출액", "영업이익", "당기순이익", "영업이익률", "순이익률", "ROE(지배주주)", "부채비율", "당좌비율", "유보율", 
                                            "EPS(원)", "PER(배)", "BPS(원)", "PBR(배)", "주당배당금(원)", "시가배당률(%)", "배당성향(%)"]:
            cells = row.find_all('td')
            if cells:
                cleaned_cells = [re.sub(r'\s+', ' ', cell.get_text().strip().replace('WnWt', '').replace('\xa0', ' ')) for cell in cells]
                data[header.get_text()] = cleaned_cells
                if not titles:  # 타이틀 한 번만 저장
                    titles = [title.get_text() for title in cells]

    df = pd.DataFrame(data, index=titles)
    df = df.replace(r'\n|\t', ' ', regex=True)

    df.columns = [col.strip() for col in df.columns]
    df.index = df.index.str.strip()
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    try:
        status = soup.find('p', class_='date').get_text()
    except AttributeError:
        status = "상태 정보를 찾을 수 없습니다."

    return df, status

In [8]:
get_finance_info("005930")
df, status = get_finance_info("005930")
#print(df)
#print(status)

(                 매출액     영업이익    당기순이익  영업이익률   순이익률 ROE(지배주주)   부채비율    당좌비율  \
 2,796,048  2,796,048  516,339  399,074  18.47  14.27     13.92  39.92  196.75   
 3,022,314  3,022,314  433,766  556,541  14.35  18.41     17.07  26.41  211.68   
 2,589,355  2,589,355   65,670  154,871   2.54   5.98      4.15  25.36  189.46   
 3,086,553  3,086,553  389,893  360,481  12.63  11.68      9.43                  
 704,646      704,646   43,061  238,414   6.11  33.84     17.07  26.41  211.68   
 637,454      637,454    6,402   15,746   1.00   2.47     13.71  26.21  210.35   
 600,055      600,055    6,685   17,236   1.11   2.87     10.66  24.80  209.73   
 674,047      674,047   24,335   58,442   3.61   8.67      9.27  24.89  205.30   
 677,799      677,799   28,247   63,448   4.17   9.36      4.15  25.36  189.46   
 724,415      724,415   50,639   48,120   6.99   6.64                            
 
                  유보율 EPS(원)  PER(배)  BPS(원) PBR(배) 주당배당금(원) 시가배당률(%) 배당성향(%)  
 2,796,048  33,1

In [10]:
df

,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%)
"2,796,048","2,796,048","516,339","399,074",18.47,14.27,13.92,39.92,196.75,"33,143.62","5,777",13.55,"43,611",1.80,"1,444",1.84,25.00
"3,022,314","3,022,314","433,766","556,541",14.35,18.41,17.07,26.41,211.68,"38,144.29","8,057",6.86,"50,817",1.09,"1,444",2.61,17.92
"2,589,355","2,589,355","65,670","154,871",2.54,5.98,4.15,25.36,189.46,"39,114.28","2,131",36.84,"52,002",1.51,"1,444",1.84,67.78
"3,086,553","3,086,553","389,893","360,481",12.63,11.68,9.43,,,,"5,086",15.22,"55,820",1.39,"1,505",,
"704,646","704,646","43,061","238,414",6.11,33.84,17.07,26.41,211.68,"38,144.29","3,460",6.86,"50,817",1.09,361,0.65,10.44
"637,454","637,454","6,402","15,746",1.00,2.47,13.71,26.21,210.35,"38,025.67",206,9.66,"51,529",1.24,361,0.56,175.00
"600,055","600,055","6,685","17,236",1.11,2.87,10.66,24.80,209.73,"38,184.87",228,13.78,"51,385",1.41,361,0.50,158.51
"674,047","674,047","24,335","58,442",3.61,8.67,9.27,24.89,205.30,"38,609.91",810,14.54,"52,068",1.31,361,0.53,44.57
"677,799","677,799","28,247","63,448",4.17,9.36,4.15,25.36,189.46,"39,114.28",887,36.84,"52,002",1.51,361,0.46,40.72
"724,415","724,415","50,639","48,120",6.99,6.64,,,,,708,116.38,,,,,


In [11]:
selected_rows = df.iloc[[2, 3, 8, 9]]
selected_rows

,매출액,영업이익,당기순이익,영업이익률,순이익률,ROE(지배주주),부채비율,당좌비율,유보율,EPS(원),PER(배),BPS(원),PBR(배),주당배당금(원),시가배당률(%),배당성향(%)
"2,589,355","2,589,355","65,670","154,871",2.54,5.98,4.15,25.36,189.46,"39,114.28","2,131",36.84,"52,002",1.51,"1,444",1.84,67.78
"3,086,553","3,086,553","389,893","360,481",12.63,11.68,9.43,,,,"5,086",15.22,"55,820",1.39,"1,505",,
"677,799","677,799","28,247","63,448",4.17,9.36,4.15,25.36,189.46,"39,114.28",887,36.84,"52,002",1.51,361,0.46,40.72
"724,415","724,415","50,639","48,120",6.99,6.64,,,,,708,116.38,,,,,


***영업이익 high, PER low. ROE high.부채비율 low***
- 안정성
:부채비율, 당좌비율, 유보율

- 수익성
:PER 영업이익 ROE 배당성향

## 섹터별 추천
- 섹터별 ETF 목록
: 정보기술, 헬스케어, 소비재, 금융, 커뮤니케이션서비스, 산업재, 에너지, 철강, 건설
- 주식 정보
: 에너지, 소재, 유틸리티, 정보기술, 건설, 헬스케어, 금융, 산업재, 소비재